In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torchvision.transforms as transforms

import librosa 
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt 

from glob import glob 
from tqdm.notebook import tqdm 

torch.manual_seed(7777)
np.random.seed(7777)

domain = ['source', 'target']
class_names = ['ToyTrain', 'gearbox', 'ToyCar', 'bearing', 'valve', 'fan', 'slider']

In [2]:
torch.cuda.is_available()

True

In [3]:
# Make custom dataset 
class CustomDataset(Dataset): 
    def __init__(self, pkl_path, transform=None, domain=0):
        self.transform = transform
        # self.path = glob("../data/unziped/*/train/*.wav")
        # print("Found file counts :", len(self.path))

        # Loading raw data
        print("="*20, "Loading data", "="*20)
        # self.audio_data = [[librosa.load(i, sr=16e3)[0], i.split("/")[3]] for i in tqdm(self.path)]
        with open(pkl_path, 'rb') as f:
            self.raw_data = pkl.load(f)
        print(".pkl data has been loaded")
        print("Loaded data path :", pkl_path)

        # Resizeing feature 
        print("Resizeing image ...") 
        self.crop_sec = 4 
        self.audio_data = [np.array(i[0][:int(16e3) * self.crop_sec]) for i in tqdm(self.raw_data) if i[2] == domain] 

        # Feature extraction 
        print("="*20, "Extracting features", "="*20) 
        self.feature_data = [librosa.feature.mfcc(y = i, sr = 16e3, n_mfcc=128,) for i in tqdm(self.audio_data)] 

        # Label 
        self.label_data = [[i[1]] for i in self.raw_data if i[2] == domain] 

    # Data size return 
    def __len__(self): 
        self.feature_len = len(self.feature_data)
        self.label_len = len(self.label_data)

        if self.feature_len == self.label_len: return self.feature_len
        else: return print("Some thing wrong with data")

    # def __size__(self): return len(self.img_data)


    # Index to data mapping 
    def __getitem__(self, idx): 
        x = torch.FloatTensor(self.feature_data[idx]).unsqueeze(0)
        resize_transform = transforms.Resize((128, 128))
        x = resize_transform(x)
        y = torch.LongTensor(self.label_data[idx])
        # y = self.label_data[idx]
        return x, y

In [5]:
dataset = CustomDataset(
    pkl_path = "./data/raw_samples/train_sr_16e3.pkl", 
)

==================== Loading data ====================
.pkl data has been loaded
Loaded data path : ./data/raw_samples/train_sr_16e3.pkl
Resizeing image ...


  0%|          | 0/7000 [00:00<?, ?it/s]

==================== Extracting features ====================


  0%|          | 0/6930 [00:00<?, ?it/s]

In [15]:
train_loader = DataLoader( 
    dataset, 
    batch_size=4, 
    shuffle=True, 
) 

In [7]:
# Define model 
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(8, 4, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(4, 8, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(8, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )
         
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [11]:
# # Initialize the autoencoder
model = Autoencoder()

# Auto config model to fit w/ gpu or cpu 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)

# Optimizer and Loss function 
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# model fit 
num_epochs = 50
for epoch in range(num_epochs):
    for img, _ in tqdm(train_loader):
        # img, _ = data
        img = img.to(device)
        # img = img.permute(1, 0, 2, 3)

        # FP 
        optimizer.zero_grad()
        output = model(img)
        loss = criterion(output, img)

        # BP
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# Save the model
# torch.save(model.state_dict(), 'conv_autoencoder.pth')

cuda


  0%|          | 0/1733 [00:00<?, ?it/s]

/home/tori/anaconda3/envs/torch/lib/python3.9/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4, 1, 128, 128])) that is different to the input size (torch.Size([4, 3, 128, 128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


KeyboardInterrupt: 

In [148]:
train_loader

In [18]:
epoch = 50
# model = Autoencoder()
model = Autoencoder()

# Auto config model to fit w/ gpu or cpu 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Training on :", device)
model.to(device)

# Model fit
print("Start training ...")
print("="*50)
v_acc = []
v_loss = []

# Optimizer and Loss function 
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch_count in range(epoch):
    avg_cost = 0
    total_batch = len(train_loader)
    

    preds = []
    print("Epoch:", epoch_count + 1)
    for img, _ in tqdm(train_loader):
        # Make prediction for loss calculation
        img = img.to(device)
        pred = model(img)
        
        # Loss caculation
        loss = criterion(pred, img)

        # Run through optimizer
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_cost += loss / total_batch

        # Accuracy calculation
        # pred = torch.argmax(pred, 1) == img
        preds.append(pred.float().mean())

    # Accuracy calculation for batch
    preds = torch.tensor(preds)
    acc = preds.float().mean()

    # Save for later training metric visualizations
    v_acc.append(acc)
    v_loss.append(float(avg_cost))

    # Visualize the results
    print("Accuracy:", acc.item() * 100)
    # print("Loss:", str(float(avg_cost)).format(":e"))
    print("Loss:", loss.item())
    print("="*50)

Training on : cuda
Start training ...
Epoch: 1


  0%|          | 0/1733 [00:00<?, ?it/s]

Accuracy: 16.816356778144836
Loss: 895.2353515625
Epoch: 2


  0%|          | 0/1733 [00:00<?, ?it/s]

Accuracy: 22.346441447734833
Loss: 1594.621826171875
Epoch: 3


...

Accuracy: 24.19225573539734
Loss: 430.5018005371094
Epoch: 12


  0%|          | 0/1733 [00:00<?, ?it/s]

Accuracy: 24.188505113124847
Loss: 490.3467712402344
Epoch: 13


  0%|          | 0/1733 [00:00<?, ?it/s]

KeyboardInterrupt: 